## 函数的复合

假设有两个函数，分别实现了加一操作和倍乘操作

In [1]:
def add_one(num):
  return num + 1


def double(num):
  return 2 * num

那想要对一个数进行先加一后翻倍的操作，我们会执行

In [2]:
double(add_one(3.14159))

8.28318

如果我们需要反复调用这个过程，往往会实现一个辅助函数。

In [3]:
def complex_calculate(num):
  return double(add_one(num))


complex_calculate(3.14159)

8.28318

假设需求发生了变化，加一操作改为加二，上面实现方式会体现一些劣势：

- 函数complex_calculate对改动较敏感，需要找到并修改add_one调用
- 更改后需要回归测试

针对这些问题，我们一般会实现一个通用的复合过程“串联”各个函数，来取代自己编写“辅助函数”

In [4]:
def pipe(arg, func1, func2):
  return func2(func1(arg))


pipe(3.14159, add_one, double)

8.28318

pipe函数的一个小问题是，不方便给它定义一个“别名”（如complex_calculate）。为了能得到一个具名的“快捷方式”，我们希望这个“复合”过程返回一个类似函数的东西，而不是直接产生结果。

利用前一节末尾我们学习的OO模拟函数知识，可以做如下的实现：

In [5]:
class Composed:
  def __init__(self, func1, func2):
    self.func1 = func1
    self.func2 = func2
  
  def __call__(self, num):
    return self.func2(self.func1(num))


complex_calculate = Composed(add_one, double)
complex_calculate(3.14159)

8.28318

在Python中，函数是一等公民（first class），可以像普通变量一样被动态创建和返回。于是上面的实现可以被简化：

In [6]:
def compose(func1, func2):
  def composed(num):
    return func2(func1(num))
  return composed


complex_calculate = Composed(add_one, double)
complex_calculate(3.14159)

8.28318

## 实际使用

仔细阅读前一节实现的各个format函数，会发现函数并不太容易测试：

- 函数体内会访问数据的属性（如data.size）
- 因此测试时必须传入一个带有对应属性（如size）的对象
- 如果对象比较难以被初始化，可能需要mocking等技术才能测试

于是我们重新设计各个format函数：

In [7]:
def format_name_v2(name):
  return name


def format_abs_size_v2(size):
  if size is None:
    return '0.0MB'

  return '{:.1f}MB'.format(size)


def format_relative_size_v2(size):
  if size is None:
    return 'Small'

  if size < 10:
    return 'Small'
  elif size < 20:
    return 'Medium'
  else:
    return 'Big'


def format_state_v2(state):
  if state is None:
    return 'Success'

  return 'Success' if state else 'Failure'


# format_date_time_v2 需要两个参数，我们目前的设计暂不支持，这里暂时忽略他

print(format_relative_size_v2(1))
print(format_relative_size_v2(11))
print(format_relative_size_v2(21))
print(format_relative_size_v2(None))


Small
Medium
Big
Small


更改后测试性提高了，但原有的格式化过程不能继续使用了。为了复用已有过程，我们增加一个中间层：

In [8]:
from typing import NamedTuple, Optional

class Data(NamedTuple):
  name: str
  size: Optional[float]
  state: Optional[bool]
  date: Optional[str]
  time: Optional[str]


def generate_data():
  yield Data("3D Objects", 35.1, True, "06/06/2020", "00:55")
  yield Data(".pylint.d", 28.85, False, "03/17/2021", "23:46")
  yield Data("Favorites", None, None, None, None)
  yield Data("temp", 19.64, True, "05/14/2020", "22:39")
  yield Data(".gitconfig", 0.74, True, "02/28/2021", "22:55")
  yield Data(".bashrc", 4.12, True, "02/28/2021", "21:55")

In [9]:
def attr_getter(attr_name):
  def getter(data):
    return getattr(data, attr_name)
  return getter


size_getter = attr_getter('size')
for d in generate_data():
  print(size_getter(d))

35.1
28.85
None
19.64
0.74
4.12


In [10]:
for d in generate_data():
  print(format_relative_size_v2(size_getter(d)))

Big
Big
Small
Medium
Small
Small


很明显上一个例子已经有了复合运算的影子，于是我们可以使用复合运算得到兼容格式化接口的函数：

In [11]:
format_name = compose(attr_getter('name'), format_name_v2)
format_abs_size = compose(attr_getter('size'), format_abs_size_v2)
format_relative_size = compose(attr_getter('size'), format_relative_size_v2)
format_state = compose(attr_getter('state'), format_state_v2)

这样前一章的格式化过程又可以使用了

In [12]:
from pprint import pprint


def generate_a_row(data):
  for f in (
    format_name,
    format_abs_size,
    format_relative_size,
    format_state,
  ):
    yield f(data)


pprint(list(
  map(
    tuple,
    map(generate_a_row, generate_data())
  )
))

[('3D Objects', '35.1MB', 'Big', 'Success'),
 ('.pylint.d', '28.9MB', 'Big', 'Failure'),
 ('Favorites', '0.0MB', 'Small', 'Success'),
 ('temp', '19.6MB', 'Medium', 'Success'),
 ('.gitconfig', '0.7MB', 'Small', 'Success'),
 ('.bashrc', '4.1MB', 'Small', 'Success')]


## 已有工具

- Python的operator库已经提供了`attrgetter`，我们可以直接使用它替代我们自己实现的`attr_getter`函数；
- toolz包提供了`compose`函数，我们可以使用它替代我们实现的相同函数，需要注意toolz的compose函数参数定义与我们的相反，它把外层函数写在前，内层函数写在后；
- toolz包也有pipe函数，不过我们暂时用不到它。

In [13]:
import operator
import toolz


# attrgetter也不支持同时获取多个属性，为了实现format_date_time_v2，我们增加一个辅助函数
def get_date_time(data):
  return data.date, data.time


def format_date_time_v2(date_and_time):
  date = date_and_time[0]
  time = date_and_time[1]
  if date is None or time is None:
    return 'Unknown'

  return date + ' ' + time


format_name = toolz.compose(format_name_v2, operator.attrgetter('name'), )
format_abs_size = toolz.compose(format_abs_size_v2, operator.attrgetter('size'))
format_relative_size = toolz.compose(format_relative_size_v2, operator.attrgetter('size'))
format_state = toolz.compose(format_state_v2, operator.attrgetter('state'))
format_date_time = toolz.compose(format_date_time_v2, get_date_time)


def generate_a_row(data):
  for f in (
    format_name,
    format_abs_size,
    format_relative_size,
    format_state,
    format_date_time,
  ):
    yield f(data)


generate_a_row_tuple = toolz.compose(tuple, generate_a_row)

pprint(list(
  map(
    generate_a_row_tuple, generate_data()
  )
))

[('3D Objects', '35.1MB', 'Big', 'Success', '06/06/2020 00:55'),
 ('.pylint.d', '28.9MB', 'Big', 'Failure', '03/17/2021 23:46'),
 ('Favorites', '0.0MB', 'Small', 'Success', 'Unknown'),
 ('temp', '19.6MB', 'Medium', 'Success', '05/14/2020 22:39'),
 ('.gitconfig', '0.7MB', 'Small', 'Success', '02/28/2021 22:55'),
 ('.bashrc', '4.1MB', 'Small', 'Success', '02/28/2021 21:55')]


## 补充知识

toolz还提供了一个juxt函数，和compose类似，juxt接受多个函数作为入参，返回一个新函数；和compose不同的是，juxt是“并联”各个函数。

![juxt](https://drek4537l1klr.cloudfront.net/borgatti/v-30/Figures/11-juxtaposing-fns.png)

利用juxt，我们可以优化一下最后的格式化过程。

In [14]:
# 我们即将取代以下过程
# def generate_a_row(data):
#   for f in (
#     format_name,
#     format_abs_size,
#     format_relative_size,
#     format_state,
#     format_date_time,
#   ):
#     yield f(data)

import toolz

juxt_a_row = toolz.juxt(
  format_name,
  format_abs_size,
  format_relative_size,
  format_state,
  format_date_time,
)

pprint(list(
  map(juxt_a_row, generate_data())
))

[('3D Objects', '35.1MB', 'Big', 'Success', '06/06/2020 00:55'),
 ('.pylint.d', '28.9MB', 'Big', 'Failure', '03/17/2021 23:46'),
 ('Favorites', '0.0MB', 'Small', 'Success', 'Unknown'),
 ('temp', '19.6MB', 'Medium', 'Success', '05/14/2020 22:39'),
 ('.gitconfig', '0.7MB', 'Small', 'Success', '02/28/2021 22:55'),
 ('.bashrc', '4.1MB', 'Small', 'Success', '02/28/2021 21:55')]
